In [1]:
# !pip3 install -r ../requirements.txt

In [2]:
import sys
import os

# プロジェクトのルートディレクトリを指定
project_root = os.path.abspath(os.path.join(os.getcwd(), './..'))
sys.path.append(project_root)

## YAML

In [3]:
from omegaconf import OmegaConf
from config.modifier import dynamically_modify_config

config_path = './../config/config.yaml'
config = OmegaConf.load(config_path)
dynamically_modify_config(config=config)

## 環境

In [4]:
from src.build import build_env

env = build_env(config=config)
max_speed = env.v_max
max_steer = env.s_max

In [ ]:
## Actor

In [ ]:
import torch
from modules.sac_module import SACModule
model = SACModule(env=env, config=config)

ckpt_path = ''
ckpt = torch.load(ckpt_path)
model.load_state_dict(ckpt['state_dict'])
actor = model.actor
actor.eval()

In [7]:
import numpy as np
from src.agent import convert_action, convert_obs

obs = env.reset()
env.render()
state = convert_obs(obs)
done = False
# シミュレーションループ
while not done:
    actions = []
    state = torch.tensor(state, dtype=torch.float32)
    nn_action, _ = actor(state)
    action = convert_action(nn_action.cpu().numpy(), max_steer=max_steer, max_speed=max_speed)
    actions.append(action)
    new_obs, step_reward, done, info = env.step(np.array(actions))
    new_state = convert_obs(new_obs)

    obs = new_obs
    state = new_state
    env.render(mode='human')
    